In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [81]:
#get data from git hub
url = 'https://raw.githubusercontent.com/narquette/NoShowAppointments/master/NoShowAppointments/Data/Preprocessing/KaggleV2-May-2016.csv?token=AFDCFMVBJY7H3IMWVN4HC626JNECG'
appt_data = pd.read_csv(url,parse_dates=['ScheduledDay','AppointmentDay'])


HTTPError: ignored

In [82]:
#explore data
appt_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null datetime64[ns, UTC]
AppointmentDay    110527 non-null datetime64[ns, UTC]
Age               110527 non-null int64
Neighborhood      110527 non-null object
Scholarship       110527 non-null int64
Hypertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMSReceived       110527 non-null int64
NoShow            110527 non-null int64
dtypes: datetime64[ns, UTC](2), float64(1), int64(9), object(2)
memory usage: 11.8+ MB


In [83]:
#change column name
appt_data.rename(columns={"Hipertension": "Hypertension"
                          ,"SMS_received": "SMSReceived"
                          ,"No-show": "NoShow"
                          ,"Neighbourhood": "Neighborhood" }
                 ,inplace=True)
appt_data.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighborhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handcap,SMSReceived,NoShow
0,2.987250e+13,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,0
1,5.589978e+14,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,0
2,4.262962e+12,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,0
3,8.679512e+11,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0
4,8.841186e+12,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,0


In [84]:
#show patientid
appt_data['PatientId'].head()

0    2.987250e+13
1    5.589978e+14
2    4.262962e+12
3    8.679512e+11
4    8.841186e+12
Name: PatientId, dtype: float64

In [85]:
#explore missing values for object data types
appt_data.select_dtypes(include='object').isnull().sum()

Gender          0
Neighborhood    0
dtype: int64

In [86]:
#set NoShow to a number value
appt_data['NoShow'].replace('No',0,inplace=True)
appt_data['NoShow'].replace('Yes',1,inplace=True)

#explore missing values for object data types
appt_data.select_dtypes(include='number').isnull().sum()

PatientId        0
AppointmentID    0
Age              0
Scholarship      0
Hypertension     0
Diabetes         0
Alcoholism       0
Handcap          0
SMSReceived      0
NoShow           0
dtype: int64

In [87]:
#look at gender counts
appt_data['Gender'].value_counts()

F    71840
M    38687
Name: Gender, dtype: int64

In [88]:
appt_data['Age'].value_counts()

 0      3539
 1      2273
 52     1746
 49     1652
 53     1651
        ... 
 115       5
 100       4
 102       2
 99        1
-1         1
Name: Age, Length: 104, dtype: int64

In [89]:
appt_data['Neighborhood'].value_counts()

JARDIM CAMBURI                 7717
MARIA ORTIZ                    5805
RESISTÊNCIA                    4431
JARDIM DA PENHA                3877
ITARARÉ                        3514
                               ... 
ILHA DO BOI                      35
ILHA DO FRADE                    10
AEROPORTO                         8
ILHAS OCEÂNICAS DE TRINDADE       2
PARQUE INDUSTRIAL                 1
Name: Neighborhood, Length: 81, dtype: int64

In [92]:
#get unique Neighborhood values
len(appt_data['Neighborhood'].unique())

81

In [90]:
# making boolean series for a team name 
no_show = appt_data['NoShow']==1

# group data by no show 
appt_data[no_show].groupby('PatientId').sum()[['NoShow']]


,NoShow
PatientId,
5.628261e+06,1
6.485121e+07,1
8.799645e+07,1
1.224513e+08,2
1.421333e+08,1
...,...
9.988130e+14,1
9.994717e+14,1
9.994792e+14,2


In [0]:
# build function to determine if the appointment is the same day
def same_day_appt(sched_day,appt_day):
    is_same_day = 0
    if sched_day.date() == appt_day.date():
      is_same_day = 1
    return is_same_day

# return the day name for the appointment day
def appt_day_name(appt_day):
    return appt_day.day_name()

# return the time of day when the appointments was scheduled
def scheduled_time_of_day(date):
  time = ""
  if date.hour < 12:
    time = "Morning" 
  elif 12 <= date.hour <= 15 :
    time = "Afternoon"
  else:
    time = "Evening"
  return time

# run same_day_appt function against ScheduledDay and AppointmentDay
appt_data['SameDayAppointment'] = appt_data.apply(lambda x: same_day_appt(x['ScheduledDay'],x['AppointmentDay']),axis=1)

# run another apply statement to determine the day of the week
appt_data['DayOfWeek'] = appt_data.apply(lambda x: appt_day_name(x['AppointmentDay']),axis=1)

# population a new column for the scheduled time of the day
appt_data['ScheduledTimeOfDay'] = appt_data.apply(lambda x: scheduled_time_of_day(x['ScheduledDay']),axis=1)


In [153]:
# view Same Day Appointment Data
appt_data['ScheduledTimeOfDay'].head(10)

0      Evening
1      Evening
2      Evening
3      Evening
4      Evening
5      Morning
6    Afternoon
7    Afternoon
8      Morning
9    Afternoon
Name: ScheduledTimeOfDay, dtype: object

In [154]:
appt_data['ScheduledTimeOfDay'].value_counts()

Morning      68481
Afternoon    31664
Evening      10382
Name: ScheduledTimeOfDay, dtype: int64

In [0]:
appt_data = pd.get_dummies(appt_data)

In [156]:
appt_data.columns

Index(['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay', 'Age',
       'Scholarship', 'Hypertension', 'Diabetes', 'Alcoholism', 'Handcap',
       ...
       'DayOfWeek_Wednesday', 'DayOfWeek_Friday', 'DayOfWeek_Monday',
       'DayOfWeek_Saturday', 'DayOfWeek_Thursday', 'DayOfWeek_Tuesday',
       'DayOfWeek_Wednesday', 'ScheduledTimeOfDay_Afternoon',
       'ScheduledTimeOfDay_Evening', 'ScheduledTimeOfDay_Morning'],
      dtype='object', length=111)

'evening'